# 🏡 Airbnb Rio de Janeiro - Predicting Daily Rates


![Jupyter](https://2.bp.blogspot.com/-RThEwEqQrtc/Uk2l9uuty4I/AAAAAAAAnE0/ZsPTiyseI_A/s1600/Rio-de-Janeiro.jpg)

## 📋 Introdução

> Este projeto tem como objetivo prever o valor das diárias de imóveis listados no Airbnb na cidade do Rio de Janeiro, utilizando características como localização, tamanho, número de quartos, e avaliações de hóspedes. Ao final, será desenvolvido um modelo preditivo que poderá ser utilizado por anfitriões para ajustar o preço de seus imóveis de acordo com a demanda e as características da propriedade.

> Para conduzir este projeto de forma estruturada e eficiente, será utilizado o framework CRISP-DM (Cross-Industry Standard Process for Data Mining), seguindo as seguintes etapas:

1. **Entendimento do Negócio 🏢:** Compreender o contexto do Airbnb e a importância de prever o preço correto para as diárias.
   
2. **Entendimento dos Dados 🔍:** Explorar e analisar os datasets de hospedagem disponíveis.
3. **Preparação dos Dados 🛠️:** Limpar, transformar e preparar os dados para modelagem.
4. **Modelagem 🤖:** Treinar modelos de machine learning para prever o valor das diárias.
5. **Avaliação 📊:** Avaliar o desempenho dos modelos.
6. **Implantação 🚀:** Desenvolver uma aplicação interativa em Streamlit.
   
> Vale ressaltar que esta é a parte do projeto focada na ciência de dados, responsável por análise, modelagem e predição. Paralelamente, o projeto também conta com uma estrutura robusta de engenharia de dados, que está sendo conduzida por um colega, encarregado de preparar e otimizar o pipeline de dados.

In [2]:
#Principais bibliotecas para exploração de dados
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pathlib
%matplotlib inline
import seaborn as sn
from ydata_profiling import ProfileReport
import ipywidgets as widget

In [66]:
#Leitura dos DF

# criando um dicionario para utilizar na criação da feature mes que sera lida no arquivo .csv
meses = {'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4, 'mai': 5, 'jun': 6,
      'jul': 7, 'ago': 8, 'set': 9, 'out': 10,'nov': 11, 'dez':12}

#Adicionando o caminho da pasta na variavel repositorio
repositorio = pathlib.Path('C:/Users/jonat/OneDrive/Área de Trabalho/Airbnb bases')

#criando uma lista para armazenar os dfs que seram apendados durante o loop
dfs = []

for arquivo in repositorio.iterdir():
    nome_mes = arquivo.name[:3]
    mes = meses[nome_mes]
    ano = arquivo.stem[-4:]
    print(ano, end=" ")
    
    df = pd.read_csv(repositorio / arquivo.name, low_memory=False)
    df['ano'] = ano
    df['mes'] = mes
    dfs.append(df)
    
df_final = pd.concat(dfs, ignore_index=True)

2018 2019 2020 2018 2019 2018 2019 2019 2020 2019 2020 2018 2019 2019 2018 2019 2020 2019 2020 2019 2018 2018 2019 2018 2019 

In [67]:
df_final.shape

(902210, 108)

#### 1° Analise exploratoria:

> Como o DF possui 108 colunas, vou extrair as 1000 primeiras linhas do DF para selecionar apenas as colunas relevantes para modelagem dos dados

In [69]:
#Salvando as 100 primeiras linhas para analisar o conteudo das colunas (identificação de colunas com dados duplicados)
df_final.head(1000).to_excel('C:/Users/jonat/Predictive_analysis_Airbnb_property_rental/Results/100_primeiros_registros.xlsx', index=False)

In [70]:
#separando as colunas que identifiquei como mais relevantes para analise do modelo 
colunas = ['id','host_response_time','host_response_rate','host_is_superhost',
           'host_listings_count','latitude','longitude','property_type','room_type',
           'accommodates','bathrooms','bedrooms','beds','bed_type','amenities','price',
           'security_deposit','cleaning_fee','guests_included','extra_people','minimum_nights',
           'maximum_nights','number_of_reviews','review_scores_rating','review_scores_accuracy',
           'review_scores_cleanliness','review_scores_checkin','review_scores_communication',
           'review_scores_location','review_scores_value','instant_bookable',
           'is_business_travel_ready','cancellation_policy','ano','mes']

#Filtrando o df apenas com apenas as colunas selecionadas
df_final = df_final.loc[:, colunas]

In [34]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 902210 entries, 0 to 902209
Data columns (total 35 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           902210 non-null  int64  
 1   host_response_time           500367 non-null  object 
 2   host_response_rate           500364 non-null  object 
 3   host_is_superhost            901750 non-null  object 
 4   host_listings_count          901750 non-null  float64
 5   latitude                     902210 non-null  float64
 6   longitude                    902210 non-null  float64
 7   property_type                902210 non-null  object 
 8   room_type                    902210 non-null  object 
 9   accommodates                 902210 non-null  int64  
 10  bathrooms                    900486 non-null  float64
 11  bedrooms                     901360 non-null  float64
 12  beds                         899708 non-null  float64
 13 

#### Tratamento de volores nulos

In [71]:
df_final.isnull().sum()

id                                  0
host_response_time             401843
host_response_rate             401846
host_is_superhost                 460
host_listings_count               460
latitude                            0
longitude                           0
property_type                       0
room_type                           0
accommodates                        0
bathrooms                        1724
bedrooms                          850
beds                             2502
bed_type                            0
amenities                           0
price                               0
security_deposit               421280
cleaning_fee                   313506
guests_included                     0
extra_people                        0
minimum_nights                      0
maximum_nights                      0
number_of_reviews                   0
review_scores_rating           448016
review_scores_accuracy         448586
review_scores_cleanliness      448413
review_score

In [72]:
percent_dados = 448551 / 902210
print(percent_dados)

0.4971691734740249


##### Lista de colunas dropadas

 > Por serem colunas com muitos valores nulos onde concentram cerca de 50% de valores nulos e por entender que estas colunas não serao totalmente relevantes para o modelo de provisão, optei por excluir.


<details>
  <summary>Clique para expandir!</summary>
  <b> <p>host_response_time: </p> </b>
  <b> <p>host_response_rate: </p> </b>
  <b> <p>security_deposit: </p> </b>
  <b> <p>cleaning_fee: </p> </b>
  <b> <p>review_scores_rating: </p> </b>
  <b> <p>review_scores_accuracy: </p> </b>
  <b> <p>review_scores_cleanliness: </p> </b>
  <b> <p>review_scores_cleanliness: </p> </b>
  <b> <p>review_scores_checkin: </p> </b>
  <b> <p>review_scores_communication: </p> </b>
  <b> <p>review_scores_value: </p> </b>

      
</detais>




In [73]:
#percorrendo o df para encontrar colunas com valores nulos acima de 300k de linhas e excluirabs

for coluna in df_final:
    if df_final[coluna].isnull().sum() > 300000:
        df_final = df_final.drop(coluna, axis=1)

#soma de volores nulos sem as colunas exluidas
df_final.isnull().sum()

id                             0
host_is_superhost            460
host_listings_count          460
latitude                       0
longitude                      0
property_type                  0
room_type                      0
accommodates                   0
bathrooms                   1724
bedrooms                     850
beds                        2502
bed_type                       0
amenities                      0
price                          0
guests_included                0
extra_people                   0
minimum_nights                 0
maximum_nights                 0
number_of_reviews              0
instant_bookable               0
is_business_travel_ready       0
cancellation_policy            0
ano                            0
mes                            0
dtype: int64

In [74]:
#Excluindo linhas vazias do DF
df_final = df_final.dropna()

df_final.shape

(897709, 24)

In [77]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 897709 entries, 0 to 902209
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id                        897709 non-null  int64  
 1   host_is_superhost         897709 non-null  object 
 2   host_listings_count       897709 non-null  int32  
 3   latitude                  897709 non-null  float64
 4   longitude                 897709 non-null  float64
 5   property_type             897709 non-null  object 
 6   room_type                 897709 non-null  object 
 7   accommodates              897709 non-null  int64  
 8   bathrooms                 897709 non-null  int32  
 9   bedrooms                  897709 non-null  int32  
 10  beds                      897709 non-null  int32  
 11  bed_type                  897709 non-null  object 
 12  amenities                 897709 non-null  object 
 13  price                     897709 non-null  float6

In [76]:
#Ajustando os tipos de dados do df

##Alterando os tipos de dados de flt para int
df_final["host_listings_count"] = df_final["host_listings_count"].astype(int)
df_final["bathrooms"] = df_final["bathrooms"].astype(int)
df_final["bedrooms"] = df_final["bedrooms"].astype(int)
df_final["beds"] = df_final["beds"].astype(int)

#Alterando os tipos de dados de obj para float subistituindo os caracteres especiais
df_final["price"] = df_final["price"].str.replace('$', '')
df_final["price"] = df_final["price"].str.replace(',', '')
df_final["price"] = df_final["price"].astype(float)

df_final["extra_people"] = df_final["extra_people"].str.replace('$', '')
df_final["extra_people"] = df_final["extra_people"].str.replace(',', '')
df_final["extra_people"] = df_final["extra_people"].astype(float)